In [3]:
import gensim, nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
from tqdm import tqdm
from sklearn import utils
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit


In [9]:
data1 = pd.read_csv(r"/Users/surabhi/Desktop/ATML_Project/InitialDataPrep-FeatureExtractionPhase-May-2020/DataPreprocessed/Preprocessed_stopword.csv")

In [10]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Content      994 non-null    object
 1   Book_Name    994 non-null    object
 2   guten_genre  994 non-null    object
 3   Author_Name  994 non-null    object
 4   id           994 non-null    object
 5   clean_data   994 non-null    object
 6   cnt          994 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 54.5+ KB


In [52]:
data1[:5]

,Content,Book_Name,guten_genre,Author_Name,id,clean_data,cnt
0,"""Love seeketh only Self to please, To bind ano...",The Narrow House,Literary,Scott| Evelyn,pg42534,bind delight joy loss ease build hell heaven ...,1
1,"To G. P. B. JAMES, Esq.\nDear James, - You, on...",Roland Cashel- Volume I (of II),Literary,Lever| Charles James,pg33468,dear time dedicated tale deep thrilling inscr...,1
2,Tokyo To Tijuana: Gabriele Departi...,Tokyo to Tijuana: Gabriele Departing America,Literary,Sills| Steven David Justin,pg12733,tokyo departing sill probable arrive city cle...,1
3,"Looking back now, after so many months of stru...",The Bad Man: A Novel,Literary,Browne| Porter Emerson| Towne| Charles Hanson,pg16968,month struggle foreboding wondered high coura...,1
4,"PAGE ""At last it was out"" See page 81 . Fronti...",The Brass Bound Box,Literary,Raymond| Evelyn,pg28509,frontispiece stretched owner floor feel queer...,1


In [11]:
train, test = train_test_split(data1, test_size=0.3, random_state=42, stratify = np.array(data1['guten_genre']))

In [13]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words= r['clean_data'], tags=[r.guten_genre]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words= r['clean_data'], tags=[r.guten_genre]), axis=1)


In [14]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [15]:
model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 695/695 [00:00<00:00, 856861.05it/s]


In [16]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 695/695 [00:00<00:00, 1650646.25it/s]


CPU times: user 19min 21s, sys: 3.09 s, total: 19min 24s
Wall time: 2min 35s


In [5]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(list(doc.words), steps=20)) for doc in sents])
    return targets, regressors

In [8]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)

NameError: name 'model_dbow' is not defined

In [118]:
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [88]:
data1['clean_data'].apply(lambda x: len(x.split(' '))).sum()

22458809

In [121]:
pd.DataFrame(list(X_train))

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.000547,0.000713,-0.001372,0.000640,0.001302,-0.001456,0.001515,0.000753,-0.000542,-0.000682,...,-0.000849,-0.001639,0.000324,0.001121,0.000387,-0.001014,-0.000720,-0.000812,-0.000798,-0.000371
1,-0.000750,0.000750,-0.000054,0.000074,-0.000920,0.000783,0.000775,-0.000442,0.001605,0.001222,...,0.000700,-0.000331,-0.001109,-0.001347,0.000891,0.001095,0.000154,-0.000135,0.000629,-0.000474
2,0.000001,0.001410,0.000221,-0.000893,0.001283,-0.000710,-0.001542,0.000631,-0.000824,-0.000375,...,-0.001603,-0.000774,-0.000947,0.001368,0.001639,-0.001062,0.001491,0.000520,0.001484,0.001588
3,-0.000528,0.001052,0.000786,-0.000715,-0.001653,-0.001436,0.001246,-0.000013,-0.000101,0.000021,...,-0.000739,-0.001595,0.000411,-0.000072,-0.001660,0.000322,0.001267,0.001424,-0.000452,0.001652
4,-0.000255,-0.001193,0.001061,-0.001293,-0.000518,0.001035,0.001034,-0.001515,-0.001646,-0.000256,...,0.001487,0.001434,0.000648,-0.001136,0.000562,-0.000939,0.000255,0.001023,-0.001264,0.001542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,-0.001378,-0.000339,-0.000418,-0.000347,-0.000372,0.000032,0.000195,0.001527,0.001648,-0.000525,...,-0.000808,0.001591,0.001025,0.000614,-0.001553,0.000574,0.000558,0.000287,-0.001560,0.000965
691,0.001223,-0.000766,-0.000499,-0.001097,0.001223,0.000951,-0.000393,0.001181,-0.000035,-0.000623,...,0.001509,0.000815,0.000429,-0.000808,0.000595,-0.000603,-0.001371,0.000073,0.001541,0.001562
692,-0.001595,0.000994,0.001624,-0.001042,0.001355,0.001465,-0.001490,0.000071,-0.000563,-0.001007,...,0.001615,0.000992,-0.001271,-0.001379,0.000692,-0.000738,0.001595,-0.001276,-0.001478,0.000671
693,0.001337,0.000326,0.001396,-0.000994,-0.000389,0.000446,-0.000937,-0.001496,0.001500,0.001397,...,-0.000045,0.000788,-0.000113,0.000404,0.001396,-0.000355,0.000031,-0.000917,-0.000497,0.000479


In [122]:
pd.DataFrame(list(X_test))

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.001031,-0.000478,-0.000320,-0.001067,-0.000375,-0.000957,0.000200,0.000994,-0.000497,-0.000649,...,0.000727,-0.000330,0.000668,0.001542,-0.001641,-0.001025,0.000894,0.001457,-0.000903,-0.001641
1,0.000172,0.000281,-0.001615,0.000227,0.001272,0.001392,-0.000168,0.000849,-0.000754,-0.001541,...,0.001625,0.000068,-0.000088,-0.000303,0.001076,0.000632,0.000358,-0.000592,0.001339,-0.001396
2,-0.000185,-0.000266,-0.001520,-0.000725,-0.001009,-0.000889,0.001329,0.000567,0.000065,0.000854,...,0.001408,0.001336,0.000929,0.001368,-0.001144,-0.001278,-0.001027,-0.000805,-0.001157,-0.000736
3,-0.000701,0.000294,0.001574,0.001537,0.000491,-0.001012,0.000061,0.000901,0.001649,0.001162,...,-0.000187,-0.000859,-0.000182,-0.001612,-0.000751,-0.000278,-0.000064,-0.000088,-0.000540,0.000217
4,0.001514,0.001050,0.000477,-0.000383,-0.000202,0.000017,0.001261,-0.000482,0.000439,-0.000950,...,-0.001151,0.000708,-0.001436,0.001275,-0.000120,-0.000867,-0.000716,0.000921,-0.001400,-0.000782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,-0.000800,-0.000878,0.000574,-0.001440,0.000338,-0.000619,-0.000467,0.000880,-0.001029,-0.000305,...,-0.000880,-0.001160,-0.000471,0.001533,-0.000901,-0.001423,-0.001030,0.000426,-0.001109,-0.000569
295,-0.000853,-0.001467,-0.001255,-0.000529,-0.000876,-0.000075,-0.000576,0.001239,0.000511,-0.001081,...,0.000838,-0.001495,-0.001516,0.000278,0.001197,0.001021,0.001158,-0.000226,-0.001134,-0.000424
296,0.000883,0.000345,-0.001111,-0.000296,-0.001503,-0.000896,0.000716,0.000422,0.000126,0.001504,...,-0.001170,-0.000982,0.000388,0.000124,-0.000118,-0.001087,0.000862,-0.001000,0.000402,-0.000312
297,0.000876,-0.001401,0.000245,0.000579,0.000234,0.000955,0.001238,-0.001380,-0.001279,0.000335,...,0.000980,-0.000366,-0.000505,-0.000951,-0.000857,-0.001331,0.000666,-0.000520,0.000583,-0.000693


In [126]:
train

,Content,Book_Name,guten_genre,Author_Name,id,clean_data,cnt
330,One day in the early June of the year 1900 I w...,The Debit Account,Literary,Onions| Oliver,pg37479,early year walk hampstead neighbourhood vale ...,1
376,A solitary room at midnight: a single wax cand...,A Whim- and Its Consequences - Collection of B...,Literary,James| G. P. R.,pg51715,solitary room midnight single candle lighted ...,1
560,"At sunrise on a first of April, there appeared...",The Confidence-Man,Literary,Melville| Herman,pg21816HermanConfidence,sunrise appeared suddenly manco capac lake ti...,1
631,"""To all such meetings as these!"" cried Densham...",Mysterious Mr. Sabin,Detective and Mystery,Oppenheim| E. Phillips,pg35661,meeting densham lifting champagne glass soft ...,1
991,"Reginald Philip Graham Thursford, Baron Traver...",The Wicked Marquis,Detective and Mystery,Oppenheim| E. Phillips,pg35361,thursford baron travers mandeleys issued morn...,1
...,...,...,...,...,...,...,...
763,"Castle Barfield, Heydon Hey, and Beacon Hargat...","Bulldog And Butterfly - From ""Schwartz"" by Dav...",Literary,Murray| David Christie,pg22273,castle barfield heydon beacon hargate form po...,1
347,"St. Cloud Hotel, New York, November, 1869.\nAt...",Wives and Widows- The Broken Life,Literary,Stephens| Ann S.,pg36374,cloud hotel york november year unconscious mi...,1
780,"""Give me where I may stand, a lever long enoug...",The Lever: A Novel,Literary,Orcutt| William Dana,pg15430,stand lever fulcrum strong archimedes girl le...,1
51,The family of Dashwood had long been settled i...,Sense and Sensibility,Literary,Austen| Jane,pg161SJaneAusSensSensi,family dashwood settled sussex estate large r...,1
